# Exploring the Impacts of Architecture and Scale on GNN Performance on Relational Data
By: Joseph Guman, Atindra Jha, and Christopher Pondoc

## Introduction
Welcome back to Relbench! In this tutorial, we'll dive a bit deeper into the benchmark + Relational Deep Learning and explore several choices around architecture, scale, and generalizability. In particular, we'll look to answer the following questions:

1. Can we train our Relational Deep Learning on one entity classification task and expect strong zero-shot performance on another entity classification task? What happens if we finetune the model?
2. How does our choice of using embedding models to generate expressive node features impact our performance on node classification tasks?
3. How can we alter and/or extend the architecture of our existing Relational Deep Learning model to improve performance on different tasks?

This notebook already assumes you've looked through the tutorials on [loading in data](https://github.com/snap-stanford/relbench/blob/main/tutorials/load_data.ipynb) and [training a model](https://github.com/snap-stanford/relbench/blob/main/tutorials/train_model.ipynb), as our walkthrough uses those guides as a launchpad to explore deeper questions. If you haven't had a chance to look through those notebooks, we suggest starting there first.

With all that being said, let's get started!

## Question 0: Why Relational Deep Learning
Before getting started, let's motivate the use of Relational Deep Learning by defining a non-GNN baseline to compare against. One of the main issues with older methods -- such as tabular methods or even just using standard statistical ML algorithms -- is that they either only work on one table or require lots of heavy feature engineering to reap the benefits of GNNs. On the other hand, Relational Deep Learning allows us to learn directly on relational data without feature engineering.

For our baseline, let's use a classical statiscal model. In particular, let's use [LightGBM](https://lightgbm.readthedocs.io/en/stable/), which is a gradient boosting framework that uses tree-based learning algorithms.

In [ ]:
# Run LightGBM baseline
from src.models.baseline import train_model

train_metrics, val_metrics, test_metrics = train_model("rel-f1", "driver-dnf")

# Print corresponding statistics
print(f"Train: {train_metrics}")
print(f"Val: {val_metrics}")
print(f"Test: {test_metrics}")

Great! It looks like just by taking together all of raw tables and merging them together, we're able to achieve reasonable baseline performance. Let's see if we can do better without having to merge together all of the tables, though, with Relational Deep Learning!

## Question 1: Can we generalize?
Let's take a look at our first question, which involves looking at whether our Relational Deep Learning model can generalize to other tasks with/without finetuning.

Let's first start by looking setting up Relbench. As with the other tutorials, we're taking a look at the `rel-f1` dataset and focusing on node classification tasks. We'll begin by training a model on the `driver-dnf` task, which predicts whether a driver will not finish a race in the next month.

In [ ]:
from src.tasks.tasks import initialize_task, db_to_graph
import torch
from torch.nn import BCEWithLogitsLoss
from torch_geometric.seed import seed_everything

# Set up dataset and task, define metrics and loss
dataset, task, train_table, val_table, test_table = initialize_task(
    "rel-f1", "driver-dnf"
)
loss_fn = BCEWithLogitsLoss()

# Set up device
seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

We can then preprocess all of our Relbench data.

In [ ]:
import os
from relbench.modeling.graph import make_pkey_fkey_graph
from torch_frame.config.text_embedder import TextEmbedderConfig
from src.embeddings.glove import GloveTextEmbedding

# Preprocess the database data and set up our text embedder
db, col_to_stype_dict = db_to_graph(dataset)
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=device), batch_size=128
)

# Load in data used to train model
root_dir = "./data"
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=os.path.join(root_dir, f"rel-f1_materialized_cache"),
)

Next, let's load in the data and have our model set up.

In [ ]:
from src.models.loader import get_loader
from src.models.rdl.graph_sage import RDLModel

# Set up data loader and model
loader_dict, entity_table = get_loader(train_table, val_table, test_table, task, data)
model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)

# if you try out different RelBench tasks you will need to change these
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
epochs = 10

Finalize, let's initialize our training run, and evaluate our model!

In [ ]:
from src.models.training import eval_model, training_run

# Get model after a training run
state_dict = training_run(
    model, device, optimizer, task, loader_dict, val_table, loss_fn, entity_table
)
model.load_state_dict(state_dict)

# Evaluate on val and test set
eval_model(model, loader_dict, "val", task, device, val_table)
eval_model(model, loader_dict, "test", task, device, None)

As we can see, we are able to roughly replicate the results from the [core Relbench paper](https://huggingface.co/spaces/relbench/leaderboard). However, do the results generalize? To do so, let's load in the data for the other entity classification task within `rel-f1` -- `driver-top3` -- and see how we do. 

In [ ]:
# Logging Results
part1_top3_labels = ["Trained on DNF, Zero-Shot on Top 3"]
part1_top3_scores = []

# Reuse functions to set up `driver-top3 task`
dataset, task, train_table, val_table, test_table = initialize_task(
    "rel-f1", "driver-top3"
)
db, col_to_stype_dict = db_to_graph(dataset)
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=os.path.join(root_dir, f"rel-f1_materialized_cache"),
)

loader_dict, entity_table = get_loader(train_table, val_table, test_table, task, data)
model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
model.load_state_dict(state_dict)

test_metrics = eval_model(model, loader_dict, "test", task, device, None)
part1_top3_scores.append(test_metrics["roc_auc"])

Unfortunately, trying out our model zero-shot does not yield amazing results. However, what happens if we use this model as a starting point for finetuning on the task? Let's experiment on fine-tuning this model with fewer epochs on the `driver-top3` task and checking its performance.

In [ ]:
# Logging Results
part1_top3_labels.append("Trained on DNF, Finetuned on Top 3")

# Get model after a training run
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
state_dict = training_run(
    model,
    device,
    optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=5,
    state_dict=state_dict,
)
model.load_state_dict(state_dict)

# Evaluate on val and test set
eval_model(model, loader_dict, "val", task, device, val_table)
test_metrics = eval_model(model, loader_dict, "test", task, device, None)
part1_top3_scores.append(test_metrics["roc_auc"])

Nice! It looks like after we finetune even after just one epoch. we're able to practically replicate the Relbench results. Finally, let's compare this approach to simply training on the task from scratch.

In [ ]:
# Logging Results
from src.models.viz import create_bar_graph
part1_top3_labels.append("Trained on Top 3 Task")

# Define a new model, don't load in old weights.
base_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
base_optimizer = torch.optim.Adam(base_model.parameters(), lr=0.005)
base_state_dict = training_run(
    base_model,
    device,
    base_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
base_model.load_state_dict(base_state_dict)

# Evaluate on val and test set
eval_model(base_model, loader_dict, "val", task, device, val_table)
test_metrics = eval_model(base_model, loader_dict, "test", task, device, None)
part1_top3_scores.append(test_metrics["roc_auc"])

# Finally, graph all of the results
create_bar_graph(part1_top3_labels, part1_top3_scores, title='ROC AUC on Top 3 Task vs. Different Training Tasks', x_label="Model", y_label="ROC AUC on Top 3 Task")

Ultimately, we don't see much of a difference from starting from random weights to using a model pre-initialized from another entity classification task.

### Challenge
Does this trend necessarily work on larger and more diverse datasets? Depending on your compute availability, try out using different datasets, like `rel-amazon`, as well as across different types of tasks!

## Question 2: Different expressiveness of node features?
Next, let's take a look at using different embedding models for node features.

The embedding models are used to help turn the tabular data into usable node features. In the Relbench tutorial, the team uses GloVe embeddings, but the paper also mentions utilizing BERT-style embeddings. In traditional NLP, BERT embeddings are much more popular given that they are contextual -- the vector representation depends on the surrounding words, compared to static embeddings used by GloVe -- and can handle words outside of their vocabulary. In addition, their embedding size is $768$ compared to GloVe's $300$, which introduces an opportunity for more expressiveness.

As an investigation, let's switch out our GloVe embedding model with BERT and retrain a new model from scratch on the `driver-dnf` task.

In [ ]:
from src.embeddings.bert import BertTextEmbedding

dataset, task, train_table, val_table, test_table = initialize_task(
    "rel-f1", "driver-dnf"
)

# Preprocess the database data and set up our text embedder
db, col_to_stype_dict = db_to_graph(dataset)
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=BertTextEmbedding(device=device), batch_size=128
)

# Load in data used to train model
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=os.path.join(root_dir, f"rel-f1_materialized_cache"),
)
loader_dict, entity_table = get_loader(train_table, val_table, test_table, task, data)

# Initialize new, untrained model using BERT embeddings
bert_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
bert_optimizer = torch.optim.Adam(bert_model.parameters(), lr=0.005)
bert_state_dict = training_run(
    bert_model,
    device,
    bert_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
bert_model.load_state_dict(bert_state_dict)

# Evaluate on val and test set
eval_model(bert_model, loader_dict, "val", task, device, val_table)
eval_model(bert_model, loader_dict, "test", task, device, None)

We ultimately don't see that drastic of a difference between using BERT embeddings and GloVe embeddings. Despite being trained differently, the fact that the models are close in size and perform similarly on [general embedding benchmarks](https://huggingface.co/spaces/mteb/leaderboard) may suggest that the results will not be that drastic. 

### Challenge
We encourage you to try larger models with even larger embedding dimensions -- to do so, use our `CustomTextEmbedding` class! To use this class, import it as below, and then specify the name of a model as used on HuggingFace:

```python
from src.embeddings.custom import CustomTextEmbedding
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=CustomTextEmbedding(model_name=<INSERT_HUGGINGFACE_MODEL_HERE>, device=device), batch_size=128
)
```

## Question 3: Different RDL model architectures?
Finally, we experiment with different RDL model architectures. In particular, we investigate what happens as we add or subtract GNN layers from our model.

### Changing the Number of GNN Layers

First, we double the number of GNN layers in our RDL pipeline, moving from `num_layers=2` to `num_layers=4`. The idea is that by adding more layers, we can create a more expressive network that can understand more complex relationships.

In [ ]:
# Define a new model, don't load in old weights.
deep_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=4,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
deep_optimizer = torch.optim.Adam(deep_model.parameters(), lr=0.005)
deep_state_dict = training_run(
    deep_model,
    device,
    deep_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
deep_model.load_state_dict(deep_state_dict)

# Evaluate on val and test set
eval_model(deep_model, loader_dict, "val", task, device, val_table)
eval_model(deep_model, loader_dict, "test", task, device, None)

As we see, training using $4$ layers actually makes the model perform worse over time. Thus, given the simplicity of the task and the size of dataset, it is likely that we are overfitting. In particular, this is because of over-smoothing. Recall that a receptive field is the set of nodes that determine the embedding of a node of interest. However, if we have too many layers, we might have it so that all node embeddings converge to the same value.

Given that using double the amount of layers leads to less optimal results, we can try the opposite strategy and halve the number of layers in the network.

In [ ]:
# Define a new model, don't load in old weights.
shallow_model = RDLModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=1,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
shallow_optimizer = torch.optim.Adam(shallow_model.parameters(), lr=0.005)
shallow_state_dict = training_run(
    shallow_model,
    device,
    shallow_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
shallow_model.load_state_dict(shallow_state_dict)

# Evaluate on val and test set
eval_model(shallow_model, loader_dict, "val", task, device, val_table)
eval_model(shallow_model, loader_dict, "test", task, device, None)

Interestingly enough, we see that even with half the number of layers, we do just about the same as with double the number of layers. Once again, this might be more task-specific as opposed to a general conclusion about GNNs and the RDL pipeline.

Finally, we try one last experiment, where we iterate the number of layers from 1 to 10.

In [ ]:
from src.models.viz import plot_roc_auc_vs_layers

# Iterate for each number of layers
layers_list = []
roc_auc_scores = []
for layers in range(1, 11):
    print(f"\nNumber of Layers = {layers}")

    # Define a new model, don't load in old weights.
    num_model = RDLModel(
        data=data,
        col_stats_dict=col_stats_dict,
        num_layers=layers,
        channels=128,
        out_channels=1,
        aggr="sum",
        norm="batch_norm",
    ).to(device)
    num_optimizer = torch.optim.Adam(num_model.parameters(), lr=0.005)
    num_state_dict = training_run(
        num_model,
        device,
        num_optimizer,
        task,
        loader_dict,
        val_table,
        loss_fn,
        entity_table,
        epochs=10,
    )
    num_model.load_state_dict(num_state_dict)

    # Evaluate on val and test set
    val_metrics = eval_model(num_model, loader_dict, "val", task, device, val_table)
    test_metrics = eval_model(num_model, loader_dict, "test", task, device, None)

    # Store the number of layers and corresponding test ROC AUC
    layers_list.append(layers)
    roc_auc_scores.append(test_metrics["roc_auc"])
plot_roc_auc_vs_layers(layers_list, roc_auc_scores)

### Challenge
Does this trend necessarily hold on larger and more diverse datasets? Depending on your compute availability, try out using different datasets, like `rel-amazon`, as well as across different types of tasks!

### What about different GNN layers?
We can also just try using different graph layers, featured in PyG. Below is GCN.

In [ ]:
# Define a new model, don't load in old weights.
from src.models.rdl.gat import RDLGATModel

gcn_model = RDLGATModel(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(device)
gcn_optimizer = torch.optim.Adam(gcn_model.parameters(), lr=0.005)
gcn_state_dict = training_run(
    gcn_model,
    device,
    gcn_optimizer,
    task,
    loader_dict,
    val_table,
    loss_fn,
    entity_table,
    epochs=10,
)
gcn_model.load_state_dict(gcn_state_dict)

# Evaluate on val and test set
eval_model(gcn_model, loader_dict, "val", task, device, val_table)
eval_model(gcn_model, loader_dict, "test", task, device, None)